In [177]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

# табличка с погодой:

Мы взяли данные о погоде в Москве (метеостанция 65518, в районе ВДНХ) в ноябре 2014 - августе 2015 с сайта https://rp5.ru/Архив_погоды_в_По . Из полученного датасета мы используем следующие признаки:

- Дата
- Температура воздуха в градусах Цельсия (T)
- Атмосферное давление в миллиметрах ртутного столба (Po)
- Относительная влажность в % (U)
- Направление ветра - категориальный признак (DD)
- Скорость ветра в метрах в секунду (Ff)
- Общая облачность (N)
- Горизонтальная дальность видимости в км (VV)
- Температура точки росы в градусах Цельсия (Td)

In [213]:
df = pd.read_csv('погода.csv', sep = ';', header = 1)[['Местное время в По', 'T', 'Po', 'U', 'DD', 'Ff', 'N', 'VV', 'Td']]
df.head(3)

,Местное время в По,T,Po,U,DD,Ff,N,VV,Td
0,31.08.2015 21:00,23.1,731.9,94.0,"Ветер, дующий с юго-юго-запада",2,"90 или более, но не 100%",10,22.1
1,31.08.2015 18:00,28.6,731.0,83.0,"Ветер, дующий с юго-юго-востока",1,"90 или более, но не 100%",10,25.4
2,31.08.2015 15:00,30.0,730.9,72.0,"Ветер, дующий с юго-юго-запада",2,"90 или более, но не 100%",10,24.4


In [179]:
df.dtypes

Местное время в По     object
T                     float64
Po                    float64
U                     float64
DD                     object
Ff                      int64
N                      object
VV                     object
Td                    float64
dtype: object

Посмотрим, какие занчения принимают категориальные признаки

In [180]:
np.unique(df.DD)

array(['Ветер, дующий с востока',
       'Ветер, дующий с востоко-северо-востока',
       'Ветер, дующий с востоко-юго-востока', 'Ветер, дующий с запада',
       'Ветер, дующий с западо-северо-запада',
       'Ветер, дующий с западо-юго-запада', 'Ветер, дующий с севера',
       'Ветер, дующий с северо-востока', 'Ветер, дующий с северо-запада',
       'Ветер, дующий с северо-северо-востока',
       'Ветер, дующий с северо-северо-запада', 'Ветер, дующий с юга',
       'Ветер, дующий с юго-востока', 'Ветер, дующий с юго-запада',
       'Ветер, дующий с юго-юго-востока',
       'Ветер, дующий с юго-юго-запада', 'Штиль, безветрие'], dtype=object)

In [181]:
np.unique(df.N)

array(['100%.', '20–30%.', '40%.', '60%.', '90  или более, но не 100%',
       'Облаков нет.'], dtype=object)

In [182]:
np.unique(df.VV)

array(['0.4', '0.7', '0.8', '0.9', '1', '1.2', '1.4', '1.8', '10', '15',
       '18', '2', '2.5', '2.8', '20', '3.2', '3.5', '4', '5', '6', '8',
       'менее 0.05'], dtype=object)

Признак N (общая облачность) упорядочить:

In [183]:
mapper = {'Облаков нет.': 0 , '20–30%.': 1, '40%.': 2, '60%.': 3,  '90  или более, но не 100%': 4, '100%.': 5}

In [184]:
df.N = df.N.apply(lambda x: mapper[x])

Признак VV (горизонтальная дальность видимости) мы решили привести к вещественному виду. Наблюдениям со значением 'менее 0.05' мы даем значение 0.05.

In [185]:
def realizer(x):
    if x.split()[0].isalpha():
        return float(x.split()[1])
    else:
        return float(x)

In [186]:
df.VV = df.VV.apply(realizer)

In [187]:
df.head()

,Местное время в По,T,Po,U,DD,Ff,N,VV,Td
0,31.08.2015 21:00,23.1,731.9,94.0,"Ветер, дующий с юго-юго-запада",2,4,10.0,22.1
1,31.08.2015 18:00,28.6,731.0,83.0,"Ветер, дующий с юго-юго-востока",1,4,10.0,25.4
2,31.08.2015 15:00,30.0,730.9,72.0,"Ветер, дующий с юго-юго-запада",2,4,10.0,24.4
3,31.08.2015 12:00,29.4,732.4,75.0,"Ветер, дующий с юго-запада",2,4,10.0,24.6
4,31.08.2015 9:00,26.5,732.7,84.0,"Ветер, дующий с юго-запада",2,4,10.0,23.6


В нашем датасате с погодой для каждого дня присутствуют наблюдения для нескольких временных точек. Мы ограничимся использованием четырех: 9:00, 12:00, 15:00, 18:00. Создадим для них отдельные таблицы

#### 9 часов:

In [188]:
df['Местное время в По'] = pd.to_datetime(df['Местное время в По'], format='%d.%m.%Y %H:%M')
df = df.rename(columns={'Местное время в По': 'date'})

# Фильтрация данных, чтобы оставить только записи со временем 09:00
df_9 = df[df['date'].dt.hour == 9]
df_9 = pd.DataFrame(df_9)
df_9['date'] = df_9['date'].dt.date

df_9.sample(3)

,date,T,Po,U,DD,Ff,N,VV,Td
131,2015-08-15,23.8,731.2,91.0,"Ветер, дующий с юго-юго-востока",1,4,10.0,22.3
1814,2015-01-06,22.0,733.8,20.0,"Ветер, дующий с севера",6,4,10.0,-1.9
226,2015-08-03,26.4,732.2,86.0,"Ветер, дующий с юга",2,4,10.0,23.8


#### 12 часов:

In [189]:
df_12 = df[df['date'].dt.hour == 12]
df_12 = pd.DataFrame(df_12)
df_12['date'] = df_12['date'].dt.date

#### 15 часов:

In [190]:
df_15 = df[df['date'].dt.hour == 15]
df_15 = pd.DataFrame(df_15)
df_15['date'] = df_15['date'].dt.date

#### 18 часов:

In [191]:
df_18 = df[df['date'].dt.hour == 18]
df_18 = pd.DataFrame(df_18)
df_18['date'] = df_18['date'].dt.date

### Соединяем данные

Мы объединяем нашу табличку с постами с табличкой с погодой

In [192]:
data = pd.read_csv('lemmatized_no_noise.csv').drop('Unnamed: 0', axis = 1)
data.sample(3)

,author,date,n_likes,n_reposts,with_photo,is_repost?,text
8384,dying,2015-01-10,537,7,False,False,весь наставать end
8824,dying,2014-12-29,350,2,False,False,любовь игнор
2322,dying,2015-06-22,978,14,False,False,писать онлайн


In [193]:
data.isna().sum()

author         0
date           0
n_likes        0
n_reposts      0
with_photo     0
is_repost?     0
text          24
dtype: int64

In [214]:
data.sample(3)

,author,date,n_likes,n_reposts,with_photo,is_repost?,text
6857,dying,2015-02-19,887,6,False,False,сколько помнить мочь забывать
2345,dying,2015-06-21,1412,31,False,False,девушка писать значит соскучиться писать значи...
932,dying,2015-07-26,391,2,False,False,хотеть


In [206]:
data.date = data.date.apply(str) # для простоты мерджа, переведем все даты в формат строки

df_9.date = df_9.date.apply(str)
df_12.date = df_12.date.apply(str)
df_15.date = df_15.date.apply(str)
df_18.date = df_18.date.apply(str)

In [207]:
merged_9 = pd.merge(left = data, right = df_9, on = 'date', how = 'left')
merged_12 = pd.merge(left = data, right = df_12, on = 'date', how = 'left')
merged_15 = pd.merge(left = data, right = df_15, on = 'date', how = 'left')
merged_18 = pd.merge(left = data, right = df_18, on = 'date', how = 'left')

In [215]:
merged_9.to_csv('data_weather_9.csv')
merged_12.to_csv('data_weather_12.csv')
merged_15.to_csv('data_weather_15.csv')
merged_18.to_csv('data_weather_18.csv')